# Experiment 005: Proper Lattice SA (egortrushin approach)

Implement the lattice-based approach from egortrushin kernel with correct parameters:
- Much longer optimization (10000+ steps)
- Smaller perturbation deltas (position_delta=0.01, angle_delta=30)
- Proper temperature scheduling
- Use Decimal precision for output

In [ ]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely.geometry import Polygon
from shapely import affinity
from shapely.strtree import STRtree
import random
import math
import copy
import time

getcontext().prec = 25
scale_factor = Decimal("1e15")

# Tree geometry
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

print("Setup complete")

In [ ]:
class ChristmasTree:
    """Christmas tree with Decimal precision for coordinates"""
    def __init__(self, center_x="0", center_y="0", angle="0"):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        self._update_polygon()
    
    def _update_polygon(self):
        # Create polygon with scaled coordinates for Shapely
        vertices = []
        for tx, ty in zip(TX, TY):
            vertices.append((float(Decimal(str(tx)) * scale_factor), 
                           float(Decimal(str(ty)) * scale_factor)))
        initial_polygon = Polygon(vertices)
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated,
            xoff=float(self.center_x * scale_factor),
            yoff=float(self.center_y * scale_factor)
        )
    
    def get_params(self):
        return self.center_x, self.center_y, self.angle
    
    def set_params(self, x, y, angle):
        self.center_x = Decimal(str(x))
        self.center_y = Decimal(str(y))
        self.angle = Decimal(str(angle)) % 360
        self._update_polygon()
    
    def clone(self):
        return ChristmasTree(str(self.center_x), str(self.center_y), str(self.angle))

def has_collision(trees):
    """Check for collisions between trees"""
    if len(trees) <= 1:
        return False
    for i, t1 in enumerate(trees):
        for j, t2 in enumerate(trees):
            if i < j:
                if t1.polygon.intersects(t2.polygon) and not t1.polygon.touches(t2.polygon):
                    return True
    return False

def calculate_score(trees):
    """Calculate score for a configuration"""
    n = len(trees)
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / float(scale_factor) for t in trees])
    side = max(xys.max(axis=0) - xys.min(axis=0))
    return side ** 2 / n

print("Classes defined")

In [ ]:
class LatticeSimulatedAnnealing:
    """Lattice-based SA following egortrushin kernel approach"""
    
    def __init__(self, trees, nt, Tmax, Tmin, nsteps, nsteps_per_T, alpha, 
                 position_delta, angle_delta, delta1, random_state=42, log_freq=100):
        self.trees = trees  # Base trees (usually 2)
        self.nt = nt  # Grid dimensions [nx, ny]
        self.Tmax = Tmax
        self.Tmin = Tmin
        self.nsteps = nsteps
        self.nsteps_per_T = nsteps_per_T
        self.alpha = alpha
        self.position_delta = position_delta
        self.angle_delta = angle_delta
        self.delta1 = delta1
        self.log_freq = log_freq
        random.seed(random_state)
    
    def perturb_tree(self, tree):
        """Perturb tree position and angle"""
        old_x, old_y, old_angle = tree.get_params()
        dx = Decimal(str(random.uniform(-self.position_delta, self.position_delta)))
        dy = Decimal(str(random.uniform(-self.position_delta, self.position_delta)))
        dangle = Decimal(str(random.uniform(-self.angle_delta, self.angle_delta)))
        new_x = old_x + dx
        new_y = old_y + dy
        new_angle = (old_angle + dangle) % 360
        tree.set_params(new_x, new_y, new_angle)
        return old_x, old_y, old_angle
    
    def translate(self, current_trees, lengthx, lengthy):
        """Create lattice by translating base trees"""
        trees_ = []
        for tree in current_trees:
            for x in range(self.nt[0]):
                for y in range(self.nt[1]):
                    new_tree = ChristmasTree(
                        str(tree.center_x + Decimal(str(x * lengthx))),
                        str(tree.center_y + Decimal(str(y * lengthy))),
                        str(tree.angle)
                    )
                    trees_.append(new_tree)
        return trees_
    
    def get_length(self, current_trees):
        """Find optimal translation distances"""
        xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / float(scale_factor) for t in current_trees])
        min_x, min_y = xys.min(axis=0)
        max_x, max_y = xys.max(axis=0)
        length = max(max_x - min_x, max_y - min_y)
        
        lengthx = length
        lengthy = length
        
        # Try to reduce lengthx
        while True:
            trees_ = self.translate(current_trees, lengthx - self.delta1, lengthy)
            if has_collision(trees_):
                break
            else:
                lengthx -= self.delta1
        
        # Try to reduce lengthy
        while True:
            trees_ = self.translate(current_trees, lengthx, lengthy - self.delta1)
            if has_collision(trees_):
                break
            else:
                lengthy -= self.delta1
        
        return lengthx, lengthy
    
    def solve(self):
        """Run simulated annealing"""
        t0 = time.time()
        T = self.Tmax
        
        current_trees = copy.deepcopy(self.trees)
        lengthx, lengthy = self.get_length(current_trees)
        trees_ = self.translate(current_trees, lengthx, lengthy)
        
        current_score = calculate_score(trees_)
        best_trees = copy.deepcopy(trees_)
        best_score = current_score
        
        print(f"Initial score: {current_score:.6f}")
        
        for step in range(self.nsteps):
            for step1 in range(self.nsteps_per_T):
                i = random.randint(0, len(current_trees) - 1)
                old_params = self.perturb_tree(current_trees[i])
                
                lengthx, lengthy = self.get_length(current_trees)
                trees_ = self.translate(current_trees, lengthx, lengthy)
                
                if has_collision(trees_):
                    current_trees[i].set_params(*old_params)
                    continue
                
                new_score = calculate_score(trees_)
                delta = new_score - current_score
                
                if delta < 0 or random.random() < math.exp(-delta / T):
                    current_score = new_score
                    if new_score < best_score:
                        best_score = new_score
                        best_trees = copy.deepcopy(trees_)
                        print(f"NEW BEST: {best_score:.6f} at step {step}")
                else:
                    current_trees[i].set_params(*old_params)
            
            # Cool down
            Tfactor = -math.log(self.Tmax / self.Tmin)
            T = self.Tmax * math.exp(Tfactor * (step + 1) / self.nsteps)
            
            if step % self.log_freq == 0:
                elapsed = time.time() - t0
                print(f"Step {step}/{self.nsteps}, T={T:.6f}, Score={current_score:.6f}, Best={best_score:.6f}, Time={elapsed:.1f}s")
        
        return best_score, best_trees

print("LatticeSimulatedAnnealing defined")

In [ ]:
# Load baseline to compare
baseline_df = pd.read_csv('/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025-csv/santa-2025.csv')

def parse_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return val[1:]
    return str(val)

def load_trees_for_n(df, n):
    prefix = f"{n:03d}_"
    rows = df[df['id'].str.startswith(prefix)]
    trees = []
    for _, row in rows.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        trees.append(ChristmasTree(x, y, deg))
    return trees

# Get baseline scores for target N values
target_ns = [72, 100, 110, 144, 156, 196, 200]
baseline_scores = {}
for n in target_ns:
    trees = load_trees_for_n(baseline_df, n)
    baseline_scores[n] = calculate_score(trees)
    print(f"N={n}: baseline score = {baseline_scores[n]:.6f}")

In [ ]:
# egortrushin kernel parameters
config = {
    "Tmax": 0.0002,
    "Tmin": 0.00005,
    "alpha": 0.99,
    "nsteps": 15,  # Number of temperature steps
    "nsteps_per_T": 500,  # Steps per temperature
    "position_delta": 0.01,  # CRITICAL: Much smaller than before
    "angle_delta": 30.0,
    "delta1": 0.01,
    "random_state": 42,
    "log_freq": 5,
}

# Grid configurations
grid_configs = {
    72: [4, 9],   # 4*9*2 = 72
    100: [5, 10], # 5*10*2 = 100
    110: [5, 11], # 5*11*2 = 110
    144: [6, 12], # 6*12*2 = 144
    156: [6, 13], # 6*13*2 = 156
    196: [7, 14], # 7*14*2 = 196
    200: [5, 20], # 5*20*2 = 200
}

print("Configuration ready")

In [ ]:
# Initialize base trees with good starting positions
# The key insight from egortrushin: start with 2 trees that interlock well

def create_initial_base_trees():
    """Create 2 base trees with good initial configuration"""
    # Two trees at 90 degree offset, positioned to interlock
    tree1 = ChristmasTree("0", "0", "0")
    tree2 = ChristmasTree("0.3", "0.2", "90")  # Offset and rotated
    return [tree1, tree2]

initial_trees = create_initial_base_trees()
print(f"Created {len(initial_trees)} base trees")
for i, t in enumerate(initial_trees):
    print(f"  Tree {i}: x={t.center_x}, y={t.center_y}, angle={t.angle}")

In [ ]:
# Test with N=72 first (smallest target)
n = 72
nt = grid_configs[n]
print(f"\n=== Testing N={n} with grid {nt} ===")
print(f"Baseline score: {baseline_scores[n]:.6f}")

# Run lattice SA
sa = LatticeSimulatedAnnealing(
    trees=initial_trees,
    nt=nt,
    **config
)

best_score, best_trees = sa.solve()

print(f"\nFinal best score: {best_score:.6f}")
print(f"Baseline score: {baseline_scores[n]:.6f}")
print(f"Improvement: {baseline_scores[n] - best_score:.6f}")

In [ ]:
# Check if we improved
if best_score < baseline_scores[n]:
    print(f"\n*** IMPROVEMENT FOUND! ***")
    print(f"Lattice score: {best_score:.6f}")
    print(f"Baseline score: {baseline_scores[n]:.6f}")
    print(f"Improvement: {baseline_scores[n] - best_score:.6f}")
else:
    print(f"\nNo improvement over baseline.")
    print(f"Lattice: {best_score:.6f}, Baseline: {baseline_scores[n]:.6f}")
    print(f"Difference: {best_score - baseline_scores[n]:.6f}")

In [ ]:
# Summary
print("\n" + "="*50)
print("PROPER LATTICE EXPERIMENT SUMMARY")
print("="*50)
print(f"Target N: {n}")
print(f"Grid: {nt}")
print(f"Lattice score: {best_score:.6f}")
print(f"Baseline score: {baseline_scores[n]:.6f}")
print(f"Gap: {best_score - baseline_scores[n]:.6f}")
print("\nNote: The baseline is already highly optimized.")
print("Lattice approach may need different initialization or longer runs.")